In [117]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn import ensemble
import seaborn as sns
import lightgbm as lgb
from sklearn import preprocessing
import matplotlib.pyplot as plt
import catboost as cb
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
import warnings
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")

In [118]:
df= pd.read_csv('rider.csv')

In [119]:
df.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [120]:
df.shape

(450000, 20)

In [121]:
df.isnull().sum()

order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                157
pickup_time               2421
delivered_time            5218
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           16948
delivered_orders         17341
cancelled                    0
undelivered_orders       17341
lifetime_order_count        53
reassignment_method     436256
reassignment_reason     436247
reassigned_order        436247
session_time              3675
cancelled_time          444782
dtype: int64

In [122]:
df[['order_date', 'order_time']] = (df.order_time.str.split(' ', expand= True))
df[['allot_date', 'allot_time']] = (df.allot_time.str.split(' ', expand= True))
df[['accept_date', 'accept_time']] = (df.accept_time.str.split(' ', expand= True))
df[['pickup_date', 'pickup_time']] = (df.pickup_time.str.split(' ', expand= True))
df[['delivered_date', 'delivered_time']] = (df.delivered_time.str.split(' ', expand= True))

In [123]:
df = df.drop(['delivered_time','pickup_time'],axis = 1)
df = df.drop(['accept_date','allot_date','pickup_date','delivered_date','cancelled_time'],axis = 1)

In [124]:
df.reassignment_method.fillna(value = 'null', inplace=True)

df['reassigned_order'].fillna(value = 0,inplace = True)
df['reassignment_reason'].fillna(value = 'null',inplace = True)

In [125]:
df.reassignment_reason.unique()

array(['null', 'Reassignment Request from SE portal.',
       'Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket',
       'Reassign'], dtype=object)

In [126]:
df['reassignment_reason'] = df['reassignment_reason'].map({'null':0, 'Reassignment Request from SE portal.': 1, 'Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket':2, 'Reassign':3 })

In [127]:
df['not_accepted'] = np.where(df['accept_time'].isna(), 1,0)

In [128]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['order_time'] = pd.to_datetime(df['order_time'])
df['allot_time'] = pd.to_datetime(df['allot_time'])
df['accept_time'] = pd.to_datetime(df['accept_time'])

In [129]:
df['day']= df['order_date'].dt.day_name()

df['lifetime_order_count_null'] = np.where(df['lifetime_order_count'].isna() ,1,0)

In [130]:
df.loc[df['rider_id']==12549,'delivered_orders'] = df.loc[df['rider_id']==12549,'delivered_orders'].fillna(value = 0)
df.loc[df['rider_id']==14345, 'delivered_orders']=df.loc[df['rider_id']==14345, 'delivered_orders'].fillna(value = 0)
df.loc[df['rider_id']==16749,'delivered_orders'] = df.loc[df['rider_id']==16749,'delivered_orders'].fillna(value = 0)
df.loc[df['order_id']==343888,'alloted_orders'] = df.loc[df['order_id']==343888,'alloted_orders'].fillna(value = 0)

In [131]:
df.loc[(df['lifetime_order_count'].isna() == True) & (df['alloted_orders'].isna() == True) & (df['delivered_orders'].isna() == True) & (df['session_time'].isna() == True),['lifetime_order_count','alloted_orders','delivered_orders','undelivered_orders']] = df.loc[(df['lifetime_order_count'].isna() == True) & (df['alloted_orders'].isna() == True) & (df['delivered_orders'].isna() == True) & (df['session_time'].isna() == True),['lifetime_order_count','alloted_orders','delivered_orders','undelivered_orders']].fillna(value = -1000)

df.loc[(df['lifetime_order_count'].isna() == True) & (df['alloted_orders'].isna() == True) & (df['delivered_orders'].isna() == True),['lifetime_order_count','alloted_orders','delivered_orders','undelivered_orders']] = df.loc[(df['lifetime_order_count'].isna() == True) & (df['alloted_orders'].isna() == True) & (df['delivered_orders'].isna() == True),['lifetime_order_count','alloted_orders','delivered_orders','undelivered_orders']].fillna(value = -999)

df.loc[df['lifetime_order_count'].isna() == True, 'delivered_orders'] = df.loc[df['lifetime_order_count'].isna() == True, 'delivered_orders'].fillna(value = 0)
df.loc[df['lifetime_order_count'].isna() == True, 'lifetime_order_count'] = df.loc[df['lifetime_order_count'].isna() == True, 'lifetime_order_count'].fillna(value = -998)

In [132]:
for col in ['alloted_orders','delivered_orders','session_time']:
    df[col] = df[col].fillna(df.groupby('rider_id')[col].transform('median'))

In [133]:
cols_log = ['alloted_orders','delivered_orders','session_time']
imputer_log = SimpleImputer(strategy = 'median')
train_iterimp = imputer_log.fit_transform(df[cols_log])
train_iterimp = pd.DataFrame(train_iterimp,columns = cols_log)
df = df.drop(cols_log, axis = 1)
df = pd.concat([df,train_iterimp],axis = 1)

In [134]:
df.loc[(df['delivered_orders']!=-999)&(df['delivered_orders']!=-1000),'undelivered_orders'] = df.loc[(df['delivered_orders']!=-999)&(df['delivered_orders']!=-1000),'alloted_orders']-df.loc[(df['delivered_orders']!=-999)&(df['delivered_orders']!=-1000),'delivered_orders']


In [135]:
df['total_distance'] = df['first_mile_distance']+df['last_mile_distance']
df['hour'] = df['order_time'].dt.hour
df['month'] = df['order_date'].dt.month
df['is_weekend'] = df['day'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

In [136]:
allotment_delay = []
acceptance_delay = []
for i in range(len(df.index)):
    allotment_delay.append((df.loc[i,'allot_time'] - df.loc[i,'order_time']).total_seconds())
df['allotment_delay'] = allotment_delay
for i in range(len(df.index)):
    acceptance_delay.append((df.loc[i,'accept_time'] - df.loc[i,'allot_time']).total_seconds())
df['acceptance_delay'] = acceptance_delay

In [137]:
df['ad_neg'] = np.where(df['acceptance_delay']<0 ,1,0)

In [138]:
cols_log = ['acceptance_delay']
imputer_log = SimpleImputer(strategy= 'median')
train_iterimp = imputer_log.fit_transform(df[cols_log])
df = df.drop(cols_log, axis=1)
df['acceptance_delay']= train_iterimp


In [139]:
def daypart(hour):
    if hour in [6,7,8,9,10]:
        return 1
    elif hour in [14,11,12,13]:
        return 2
    elif hour in [15,16,17,18]:
        return 3
    else: return 0
dayparts = df.hour.apply(daypart)
df['dayparts'] = dayparts

In [140]:
df = df.drop(['order_time','order_date','allot_time','accept_time'],axis = 'columns')

df['reassignment_method'] = df['reassignment_method'].map({'null':0,'auto':1,'manual':2})
df['day'] = df['day'].map({'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6})

df['undelivered_orders'] = df['undelivered_orders'].astype('int')
df['reassigned_order'] = df['reassigned_order'].astype('int')
df['undelivered_orders'] = df['undelivered_orders'].astype('int')
df['day'] = df['day'].astype('int')
df['dayparts'] = df['dayparts'].astype('int')
df['reassignment_method'] = df['reassignment_method'].astype('int')


In [141]:
def bin_last(x):
    if x>0 and x<0.45:
        return 0
    elif x>=0.45 and x<0.9:
        return 1
    elif x>=0.9 and x<1.34:
        return 2
    elif x>= 1.34 and x<1.79:
        return 3
    elif x>=1.79 and x<2.24:
        return 4
    elif x>=2.24 and x<2.69:
        return 5
    elif x>=2.69 and x<2.52:
        return 6
    elif x>= 2.52and x<3.14:
        return 7
    elif x>=3.14 and x<3.59:
        return 8
    elif x>=3.59 and x<4.03:
        return 9
    elif x>= 4.03and x<4.48:
        return 10
    elif x>=4.48 and x<4.93:
        return 11
    elif x>=4.93 and x<5.38:
        return 12
    elif x>= 5.38and x<6.27:
        return 13
    else:
        return 14
df['bin_last'] = df['last_mile_distance'].apply(bin_last)


In [142]:
numeric_cols = [   
'rider_id',                       
'first_mile_distance',            
'last_mile_distance',            
'lifetime_order_count',           
'alloted_orders',                 
'delivered_orders',               
'session_time','total_distance','acceptance_delay','allotment_delay'
]
categorical_cols = [
'rider_id',                      
'undelivered_orders',                      
'reassignment_method',           
'reassignment_reason',            
'reassigned_order',              
'day',                            
'hour',                          
'dayparts',                      
'lifetime_order_count_null','ad_neg','is_weekend','bin_last']

numeric_df = df[numeric_cols]
categorical = df[categorical_cols]

In [143]:
agg = numeric_df.groupby('rider_id').agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
agg.columns = ['rider_id','first_mile_distance_count','first_mile_distance_mean','first_mile_distance_max','first_mile_distance_min','first_mile_distance_sum',
               'last_mile_distance_count','last_mile_distance_mean','last_mile_distance_max','last_mile_distance_min','last_mile_distance_sum',
               'lifetime_order_count_count','lifetime_order_count_mean','lifetime_order_count_max','lifetime_order_count_min','lifetime_order_count_sum',
               'alloted_orders_count','alloted_orders_mean','alloted_orders_max','alloted_orders_min','alloted_orders_sum',
               'delivered_orders_count','delivered_orders_mean','delivered_orders_max','delivered_orders_min','delivered_orders_sum',
               'session_time_count','session_time_mean','session_time_max','session_time_min','session_time_sum',
               'total_distance_count','total_distance_mean','total_distance_max','total_distance_min','total_distance_sum',
               'acceptance_delay_count','acceptance_delay_mean','acceptance_delay_max','acceptance_delay_min','acceptance_delay_sum',
               'allotment_delay_count','allotment_delay_mean','allotment_delay_max','allotment_delay_min','allotment_delay_sum'
              ]
df = df.merge(agg,how = 'left', on = 'rider_id')

cat = categorical.groupby('rider_id').agg(['sum', 'mean'])
cat.columns = ['rider_id'
              'undelivered_orders_sum','undelivered_orders_mean',
               'reassignment_method_sum','reassignment_method_mean',
               'reassignment_reason_sum','reassignment_reason_mean',
              'reassigned_order_sum','reassigned_order_mean',
              'day_sum','day_mean',
              'hours_sum','hours_mean',
              'dayparts_sum','dayparts_mean',
              'lifetime_order_count_null_sum','lifetime_order_count_null_mean',
              'ad_neg_sum','ad_neg_mean','is_weekend_sum','is_weekend_mean','bin_last_sum','bin_last_mean']
df = df.merge(cat,how = 'left', on = 'rider_id')

undersample = RandomUnderSampler(sampling_strategy = 0.5,random_state = 42)

df.cancelled.value_counts()

0    444782
1      5218
Name: cancelled, dtype: int64

In [144]:
def create_folds(dfv, num_splits):
    dfv["kfold"] = -1
    dfv = dfv.sample(frac=1).reset_index(drop=True)
    kf = StratifiedKFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X=dfv.drop(['cancelled'], axis =1), y=dfv.cancelled)):
        dfv.loc[v_, 'kfold'] = f
    return dfv

In [145]:
df = create_folds(df, 3)

In [146]:
df

,order_id,rider_id,first_mile_distance,last_mile_distance,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,...,dayparts_mean,lifetime_order_count_null_sum,lifetime_order_count_null_mean,ad_neg_sum,ad_neg_mean,is_weekend_sum,is_weekend_mean,bin_last_sum,bin_last_mean,kfold
0,204268,8581,1.462800,1.20,0,0,1862.0,0,0,0,...,1.400000,0,0.0,0,0.0,0,0.000000,263,5.260000,0
1,266012,4565,1.419800,2.33,0,1,1523.0,0,0,0,...,2.204545,0,0.0,0,0.0,10,0.227273,190,4.318182,0
2,511123,13709,0.199778,5.98,0,0,206.0,0,0,0,...,2.047619,0,0.0,0,0.0,17,0.404762,229,5.452381,0
3,365572,3744,0.934000,3.75,0,1,273.0,0,0,0,...,1.697674,0,0.0,0,0.0,21,0.244186,635,7.383721,0
4,298575,586,0.526100,1.05,0,0,1294.0,0,0,0,...,2.500000,0,0.0,0,0.0,0,0.000000,61,6.100000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,488356,12503,2.082900,5.01,0,0,770.0,0,0,0,...,1.829268,0,0.0,0,0.0,11,0.268293,309,7.536585,2
449996,371248,3539,2.892200,5.76,0,0,1366.0,0,0,0,...,1.804348,0,0.0,0,0.0,27,0.293478,587,6.380435,2
449997,264803,12034,1.551100,5.77,0,0,50.0,0,0,0,...,2.928571,0,0.0,0,0.0,0,0.000000,97,6.928571,2
449998,548747,4702,0.012963,1.89,0,2,1187.0,0,0,0,...,2.692308,0,0.0,0,0.0,0,0.000000,455,7.000000,2


In [147]:
final_valid_predictions_1 = {}
final_valid_predictions_2 = {}
final_valid_predictions_3 = {}
final_valid_predictions_4 = {}
final_valid_predictions_5 = {}
final_valid_predictions_6 = {}

In [148]:
for fold in range(3):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    valid_ids = xvalid['order_id'].values.tolist()
    
    useless_features = ['order_id', 'kfold', 'cancelled']
    ytrain = xtrain['cancelled']
    yvalid = xvalid['cancelled']
    
   
    useful_features = [f for f in xtrain.columns if f not in useless_features]
    useful_features_valid = [f for f in xvalid.columns if f not in useless_features]
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features_valid]
    
    xtrain,ytrain = undersample.fit_resample(xtrain,ytrain)
    
    model_1 = ensemble.RandomForestClassifier()
    model_1.fit(xtrain, ytrain)
    preds_valid_1 = model_1.predict_proba(xvalid)[:,1]
    final_valid_predictions_1.update(dict(zip(valid_ids, preds_valid_1)))
    
    model_2 = lgb.LGBMClassifier()
    model_2.fit(xtrain,ytrain)
    preds_valid_2 = model_2.predict_proba(xvalid)[:,1]
    final_valid_predictions_2.update(dict(zip(valid_ids, preds_valid_2)))
    
    model_3 = cb.CatBoostClassifier()
    model_3.fit(xtrain, ytrain, verbose=False)
    preds_valid_3 = model_3.predict_proba(xvalid)[:,1]
    final_valid_predictions_3.update(dict(zip(valid_ids, preds_valid_3)))
    
    model_4 = XGBClassifier()      
    model_4.fit(xtrain, ytrain)
    preds_valid_4 = model_4.predict_proba(xvalid)[:,1]
    final_valid_predictions_4.update(dict(zip(valid_ids, preds_valid_4)))

In [149]:
final_valid_predictions_1 = pd.DataFrame.from_dict(final_valid_predictions_1, orient="index").reset_index()
final_valid_predictions_1.columns = ["order_id", "pred_1"]
df = df.merge(final_valid_predictions_1, on = 'order_id', how = 'left')

final_valid_predictions_2 = pd.DataFrame.from_dict(final_valid_predictions_2, orient="index").reset_index()
final_valid_predictions_2.columns = ["order_id", "pred_2"]
df = df.merge(final_valid_predictions_2, on = 'order_id', how = 'left')

final_valid_predictions_3 = pd.DataFrame.from_dict(final_valid_predictions_3, orient="index").reset_index()
final_valid_predictions_3.columns = ["order_id", "pred_3"]
df = df.merge(final_valid_predictions_3, on = 'order_id', how = 'left')

final_valid_predictions_4 = pd.DataFrame.from_dict(final_valid_predictions_4, orient="index").reset_index()
final_valid_predictions_4.columns = ["order_id", "pred_4"]
df= df.merge(final_valid_predictions_4, on = 'order_id', how = 'left')

In [151]:
logreg = LogisticRegression()
from sklearn.model_selection import train_test_split
col_temp = ['pred_1','pred_2','pred_3','pred_4']
x_temp =df[col_temp]
y_temp = df['cancelled']
X_train, X_test, y_train, y_test = train_test_split(x_temp, y_temp, test_size= 0.2, random_state= 41)
logreg.fit(X_train, y_train)
pred_temp = logreg.predict(X_test)
print(roc_auc_score(y_test, pred_temp))

0.965874
